In [1]:
from __future__ import print_function
from six.moves import cPickle as pickle
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import tensorflow as tf

In [2]:
f=open('Data.pickle','rb')
Data=open('Data.pickle','rb')
train_dataset=pickle.load(Data)['train_dataset']
Data.seek(0)
train_labels=pickle.load(Data)['train_labels']
Data.seek(0)
test_dataset=pickle.load(Data)['test_dataset']
Data.seek(0)
test_labels=pickle.load(Data)['test_labels']
Data.close()

In [3]:
count=0
Corrupted_data=np.zeros((28,28),dtype=np.float32)
for i in range(train_dataset.shape[0]):
    if np.array_equal(train_dataset[i,:,:],Corrupted_data):
        Train_data[count,:,:]=train_dataset[i,:,:]
        Train_label[count]=train_labels[i]
        count=count+1
print(count)        
print(train_labels.shape)
print(train_dataset.shape)

train_dataset=train_dataset.reshape([train_dataset.shape[0],train_dataset.shape[1],train_dataset.shape[2],1])
test_dataset=test_dataset.reshape([test_dataset.shape[0],test_dataset.shape[1],test_dataset.shape[2],1])
print(test_dataset.shape)
print(train_dataset.shape)
print(test_labels.shape)

0
(50000,)
(50000, 28, 28)
(900, 28, 28, 1)
(50000, 28, 28, 1)
(900,)


In [4]:
def One_hot_encoder(labels,C):
    C=tf.constant(C,dtype=tf.int32,name='C')
    encoded=tf.one_hot(labels,C)
    with tf.Session() as sess:
        return sess.run(encoded)

In [5]:
Train_labels=One_hot_encoder(train_labels,10)
Test_labels=One_hot_encoder(test_labels,10)
print(Train_labels.shape)

(50000, 10)


In [6]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    X = tf.placeholder(shape=(None, n_H0, n_W0, n_C0),dtype=tf.float32)
    Y = tf.placeholder(shape=(None, n_y),dtype=tf.float32)
    return X, Y

In [7]:
def initialize_parameters():
    W1 = tf.get_variable('W1',[4,4,1,8],initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W2 = tf.get_variable('W2',[2,2,8,16],initializer=tf.contrib.layers.xavier_initializer(seed=0))
    parameters = {"W1": W1,
                  "W2": W2}
    return parameters

In [8]:
def forward_propagation(X, parameters):
    W1 = parameters['W1']
    W2 = parameters['W2']
    Z1 = tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool(A1,padding='SAME',ksize=[1,8,8,1],strides=[1,8,8,1])
    Z2 = tf.nn.conv2d(P1,W2,strides=[1,1,1,1],padding='SAME')
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool(A2,padding='SAME',ksize=[1,4,4,1],strides=[1,4,4,1])
    P2 = tf.contrib.layers.flatten(P2)
    Z3 = tf.contrib.layers.fully_connected(P2,10,activation_fn=None)
    return Z3

In [9]:
def compute_cost(Z3, Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y))
    return cost

In [10]:
import time as t
X, Y = create_placeholders(28,28,1,10)
parameters = initialize_parameters()
Z3 = forward_propagation(X, parameters)
cost = compute_cost(Z3, Y)
optimizer = tf.train.AdamOptimizer().minimize(cost)
init = tf.global_variables_initializer()
predict_op = tf.argmax(Z3, 1)
correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))        
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
with tf.Session() as sess:
    sess.run(init)
    print("Training Started")
    A=t.time()
    for i in range(1500):
        _ , temp_cost,acc = sess.run([optimizer,cost,accuracy],{X:train_dataset,Y:Train_labels})
        print(i," : ",temp_cost," : ",acc)
    print('Training Over time taken ',t.time()-A,'sec')
    A=t.time()
    print('Testing Accuracy',sess.run(accuracy,{X:test_dataset,Y:Test_labels}))
    print('Testing Time',t.time()-A)
    

Training Started
0  :  2.40939  :  0.09244
1  :  2.39925  :  0.0902
2  :  2.38984  :  0.08804
3  :  2.38116  :  0.08674
4  :  2.37318  :  0.086
5  :  2.36585  :  0.0864
6  :  2.35916  :  0.08578
7  :  2.35303  :  0.08698
8  :  2.34743  :  0.08894
9  :  2.34231  :  0.0921
10  :  2.33763  :  0.09632
11  :  2.33333  :  0.09918
12  :  2.32937  :  0.1033
13  :  2.32571  :  0.10588
14  :  2.3223  :  0.1095
15  :  2.31913  :  0.11224
16  :  2.31615  :  0.11398
17  :  2.31334  :  0.11496
18  :  2.31068  :  0.11492
19  :  2.30815  :  0.11594
20  :  2.30572  :  0.1166
21  :  2.30337  :  0.11698
22  :  2.30108  :  0.11718
23  :  2.29885  :  0.11692
24  :  2.29665  :  0.11688
25  :  2.29448  :  0.11662
26  :  2.29234  :  0.11756
27  :  2.2902  :  0.11808
28  :  2.28808  :  0.11986
29  :  2.28596  :  0.12216
30  :  2.28385  :  0.12486
31  :  2.28178  :  0.12906
32  :  2.27974  :  0.13306
33  :  2.27773  :  0.13704
34  :  2.27573  :  0.14206
35  :  2.27372  :  0.14704
36  :  2.27172  :  0.152
37  : 

297  :  0.542026  :  0.84802
298  :  0.539842  :  0.84842
299  :  0.537685  :  0.84872
300  :  0.535557  :  0.84916
301  :  0.533461  :  0.84946
302  :  0.531392  :  0.84978
303  :  0.529347  :  0.85038
304  :  0.527329  :  0.85072
305  :  0.525337  :  0.85112
306  :  0.52337  :  0.85162
307  :  0.521427  :  0.85194
308  :  0.519509  :  0.85242
309  :  0.517613  :  0.85272
310  :  0.515741  :  0.85312
311  :  0.513891  :  0.85338
312  :  0.512063  :  0.85368
313  :  0.510258  :  0.85408
314  :  0.508474  :  0.85426
315  :  0.506709  :  0.8546
316  :  0.504966  :  0.85504
317  :  0.503245  :  0.85556
318  :  0.501543  :  0.8558
319  :  0.49986  :  0.85614
320  :  0.498197  :  0.85638
321  :  0.496554  :  0.8568
322  :  0.494929  :  0.85728
323  :  0.493322  :  0.85762
324  :  0.491732  :  0.8578
325  :  0.490159  :  0.85782
326  :  0.488605  :  0.85822
327  :  0.487067  :  0.85884
328  :  0.485546  :  0.85908
329  :  0.484041  :  0.85956
330  :  0.482552  :  0.8598
331  :  0.481079  :  

583  :  0.30053  :  0.90772
584  :  0.300116  :  0.9078
585  :  0.299702  :  0.90792
586  :  0.29929  :  0.90794
587  :  0.298879  :  0.908
588  :  0.298469  :  0.9082
589  :  0.29806  :  0.9084
590  :  0.297652  :  0.90854
591  :  0.297247  :  0.90858
592  :  0.296842  :  0.90866
593  :  0.296437  :  0.90884
594  :  0.296034  :  0.90902
595  :  0.295633  :  0.90914
596  :  0.295232  :  0.90924
597  :  0.294833  :  0.9094
598  :  0.294436  :  0.90948
599  :  0.294039  :  0.90946
600  :  0.293644  :  0.9097
601  :  0.293249  :  0.9098
602  :  0.292855  :  0.9099
603  :  0.292463  :  0.90996
604  :  0.292072  :  0.91016
605  :  0.291683  :  0.91028
606  :  0.291295  :  0.91046
607  :  0.290908  :  0.91076
608  :  0.290523  :  0.91094
609  :  0.290139  :  0.91106
610  :  0.289756  :  0.91126
611  :  0.289374  :  0.91136
612  :  0.288993  :  0.91148
613  :  0.288614  :  0.91156
614  :  0.288235  :  0.91182
615  :  0.287858  :  0.91182
616  :  0.287482  :  0.91192
617  :  0.287107  :  0.912

869  :  0.220289  :  0.93146
870  :  0.220103  :  0.9315
871  :  0.219917  :  0.93152
872  :  0.219731  :  0.93156
873  :  0.219546  :  0.93172
874  :  0.219362  :  0.93192
875  :  0.219178  :  0.93202
876  :  0.218995  :  0.93204
877  :  0.218813  :  0.9321
878  :  0.218631  :  0.93214
879  :  0.21845  :  0.93212
880  :  0.218269  :  0.93218
881  :  0.218089  :  0.93222
882  :  0.217909  :  0.9323
883  :  0.217729  :  0.93234
884  :  0.21755  :  0.93226
885  :  0.217371  :  0.93222
886  :  0.217193  :  0.9323
887  :  0.217015  :  0.93236
888  :  0.216838  :  0.93248
889  :  0.21666  :  0.93256
890  :  0.216484  :  0.93258
891  :  0.216307  :  0.93266
892  :  0.216131  :  0.93262
893  :  0.215955  :  0.93272
894  :  0.21578  :  0.93274
895  :  0.215606  :  0.93278
896  :  0.215431  :  0.9329
897  :  0.215257  :  0.93286
898  :  0.215083  :  0.93288
899  :  0.214909  :  0.93294
900  :  0.214736  :  0.93292
901  :  0.214563  :  0.93296
902  :  0.21439  :  0.93308
903  :  0.214218  :  0.9

1150  :  0.180862  :  0.94266
1151  :  0.180754  :  0.94264
1152  :  0.180646  :  0.94268
1153  :  0.180539  :  0.94268
1154  :  0.18043  :  0.94276
1155  :  0.180323  :  0.94278
1156  :  0.180216  :  0.94276
1157  :  0.180109  :  0.94276
1158  :  0.180002  :  0.94278
1159  :  0.179896  :  0.9428
1160  :  0.179789  :  0.9428
1161  :  0.179683  :  0.94282
1162  :  0.179577  :  0.94284
1163  :  0.179471  :  0.94288
1164  :  0.179366  :  0.94296
1165  :  0.17926  :  0.94292
1166  :  0.179155  :  0.94298
1167  :  0.179049  :  0.94304
1168  :  0.178944  :  0.9431
1169  :  0.178839  :  0.94304
1170  :  0.178734  :  0.94306
1171  :  0.178629  :  0.94302
1172  :  0.178524  :  0.94302
1173  :  0.178418  :  0.9431
1174  :  0.178313  :  0.9431
1175  :  0.178208  :  0.94316
1176  :  0.178104  :  0.94328
1177  :  0.177998  :  0.94332
1178  :  0.177894  :  0.94324
1179  :  0.177789  :  0.94326
1180  :  0.177684  :  0.94326
1181  :  0.177579  :  0.94334
1182  :  0.177474  :  0.94332
1183  :  0.177369

1427  :  0.154322  :  0.95116
1428  :  0.154241  :  0.95114
1429  :  0.154159  :  0.95112
1430  :  0.154078  :  0.95106
1431  :  0.153997  :  0.95108
1432  :  0.153917  :  0.95104
1433  :  0.153837  :  0.95104
1434  :  0.153756  :  0.9511
1435  :  0.153676  :  0.9511
1436  :  0.153596  :  0.9511
1437  :  0.153516  :  0.95114
1438  :  0.153437  :  0.95112
1439  :  0.153357  :  0.95116
1440  :  0.153277  :  0.95116
1441  :  0.153197  :  0.95116
1442  :  0.153118  :  0.95118
1443  :  0.153039  :  0.9512
1444  :  0.15296  :  0.9512
1445  :  0.15288  :  0.95118
1446  :  0.152801  :  0.9512
1447  :  0.152721  :  0.95122
1448  :  0.152642  :  0.95122
1449  :  0.152562  :  0.95118
1450  :  0.152482  :  0.95122
1451  :  0.152402  :  0.95122
1452  :  0.152322  :  0.95128
1453  :  0.152241  :  0.95134
1454  :  0.152161  :  0.95142
1455  :  0.152081  :  0.95144
1456  :  0.152001  :  0.9515
1457  :  0.15192  :  0.9515
1458  :  0.151841  :  0.95166
1459  :  0.151761  :  0.95164
1460  :  0.151681  : 